In [4]:
import numpy as np
import pandas as pd

import pymc3 as pm
import theano.tensor as T
from theano.compile.ops import as_op
import theano

from IPython.display import Image
#import utils
%matplotlib inline

with pm.Model() as model:
    e1_virt_prob = np.array([
        [1, 0],
        [0, 1],
    ])
        
    probAFaulty = pm.Uniform('probAFaulty', 0, 1)
    aFaultStat = pm.Binomial('aFaultStat', n=200, p=probAFaulty)
    
    e1 = pm.Deterministic('e1', T.switch(T.eq(aFaultStat, 10), 0, 1))
    # soft evidence for probAFaulty
    e1_virt_prob_shared = theano.shared(e1_virt_prob)  # make it global
    e1_virt_prob_final = e1_virt_prob_shared[e1]  # select the prob array that "happened" thanks to parents
    e1_virt = pm.Categorical('e1_virt', p=e1_virt_prob_final, observed=0)
    
    probBFaulty = pm.Uniform('probBFaulty', 0, 1)
    bFaultStat = pm.Binomial('bFaultStat', n=200, p=probBFaulty, observed=15)
    
    paf_pbf = pm.Deterministic('paf_pbf', probAFaulty - probBFaulty)
    
    hypothesis = pm.Deterministic('hypothesis', T.switch(T.lt(paf_pbf, 0), 0, 1))
    
with model:
    SAMPLE_NUM = 10000
    trace = pm.sample(SAMPLE_NUM)
    
    pm.summary(trace, varnames=['probAFaulty'], start=1000)
    pm.traceplot(trace[1000:], varnames=['probAFaulty'])
    
    pm.summary(trace, varnames=['probBFaulty'], start=1000)
    pm.traceplot(trace[1000:], varnames=['probBFaulty'])
    
    pm.summary(trace, varnames=['paf_pbf'], start=1000)    
    pm.traceplot(trace[1000:], varnames=['paf_pbf'])
    
    pm.summary(trace, varnames=['hypothesis'], start=1000)    
    pm.traceplot(trace[1000:], varnames=['hypothesis'])
    # print(sum(trace['C3_0']) / float(SAMPLE_NUM))

Assigned NUTS to probAFaulty_interval__
Assigned Metropolis to aFaultStat
Assigned NUTS to probBFaulty_interval__
C:\Users\igbatov\AppData\Local\conda\conda\envs\pymc3\lib\site-packages\pymc3\model.py:384: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if not np.issubdtype(var.dtype, float):
  0%|                                                                                                                                      | 0/10500 [00:00<?, ?it/s]


ValueError: Bad initial energy: inf. The model might be misspecified.